In [1]:
# Modules: gdal/3.6.4  
# Environment base: /g/data/xe2/John/geospatenv

In [2]:
# Note: For the canopy_height.download_new_tiles to work, you need to create an AWS account, then create a file named .aws/credentials in your home directory (e.g. /home/147/cb8590) with these contents:
# [default]
# aws_access_key_id = ACCESS_KEY
# aws_secret_access_key = SECRET_KEY

### Imports and Functions

In [1]:
# Standard library
import os

# Local imports
os.chdir(os.path.join(os.path.expanduser('~'), "Projects/PaddockTS"))
from DAESIM_preprocess.util import *
from DAESIM_preprocess.terrain_tiles import terrain_tiles
from DAESIM_preprocess.slga_soils import slga_soils, asris_urls
from DAESIM_preprocess.ozwald_yearly import ozwald_yearly_average
from DAESIM_preprocess.ozwald_8day import ozwald_8day, ozwald_8day_abbreviations
from DAESIM_preprocess.ozwald_daily import ozwald_daily, ozwald_daily_abbreviations
from DAESIM_preprocess.silo_daily import silo_daily
from DAESIM_preprocess.canopy_height import canopy_height

# imports for the map

from ipyleaflet import Map, DrawControl, WidgetControl, GeoJSON, basemaps
from ipywidgets import Button, Layout, HTML, VBox
from shapely.geometry import Polygon
from IPython.display import display

#Code for plotting

def plot_polygon(lat,lon,buffer,stub,start_year,end_year):
    # Calculate the polygon coordinates
    min_lat = lat - buffer
    max_lat = lat + buffer
    min_lon = lon - buffer
    max_lon = lon + buffer
    
    polygon_coords = [
        [min_lon, min_lat],  # Bottom-left
        [min_lon, max_lat],  # Top-left
        [max_lon, max_lat],  # Top-right
        [max_lon, min_lat],  # Bottom-right
        [min_lon, min_lat]   # Close the polygon
    ]
    
    # Create a GeoJSON object for the polygon
    polygon_geojson = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [polygon_coords]
                },
                "properties": {"name": "Buffer Polygon"}
            }
        ]
    }
    
    # Calculate the center of the map
    center = [lat, lon]
    
    # Create the map
    m = Map(
        center=center,
        zoom=10,
        basemap=basemaps.Esri.WorldImagery,
        layout=Layout(height='800px')
    )
    
    # Add the polygon layer to the map
    polygon_layer = GeoJSON(
        data=polygon_geojson,
        style={'color': 'red', 'fillOpacity': 0.1, 'weight': 2}
    )
    m.add_layer(polygon_layer)
    
    # Create info displays with better styling
    title_output = HTML(
        value='<h3 style="margin: 0;">Selected Area Information</h3>'
    )
    
    coords_output = HTML(
        value='<div style="padding: 10px; background-color: white; border-radius: 5px;">' +
              'Draw a rectangle to get coordinates</div>'
    )
    
    # Create a container for the info displays
    info_container = VBox([
        title_output,
        coords_output
    ], layout=Layout(
        padding='10px',
        background_color='rgba(255, 255, 255, 0.9)',
        border_radius='5px'
    ))
    
    # Add the info container to the map
    info_control = WidgetControl(
        widget=info_container,
        position='bottomleft'
    )
    m.add_control(info_control)
    
    # Function to handle the drawn rectangle
    def handle_draw(self, action, geo_json):
        global lat, lon, buffer  # Declare globals
        
        if action == 'created':
            # Get the coordinates of the drawn polygon
            coords = geo_json['geometry']['coordinates'][0]
            
            # Calculate bounds
            lons = [coord[0] for coord in coords]
            lats = [coord[1] for coord in coords]
            
            min_lon, max_lon = min(lons), max(lons)
            min_lat, max_lat = min(lats), max(lats)
            
            # Calculate center and set global variables
            lat = (min_lat + max_lat) / 2
            lon = (min_lon + max_lon) / 2
            
            # Calculate buffer (using the larger dimension) and set global variable
            buffer_lat = (max_lat - min_lat) / 2
            buffer_lon = (max_lon - min_lon) / 2
            buffer = max(buffer_lat, buffer_lon)
            
            # Update the info display with better formatting
            info_text = f"""
            <div style="padding: 10px; background-color: white; border-radius: 5px;">
                <b>Center coordinates:</b><br>
                lat={lat:.6f}<br>
                lon={lon:.6f}<br>
                buffer={buffer:.6f}<br>
                <small style="color: #666;">Global variables 'lat', 'lon', and 'buffer' have been set</small><br>
                <small style="color: #666;">Click the trash icon to clear and draw again</small>
            </div>
            """
            coords_output.value = info_text
            
            # this doesn't work:
            # Print confirmation to notebook output
            # print(f"Global variables have been set:")
            # print(f"lat = {lat:.6f}")
            # print(f"lon = {lon:.6f}")
            # print(f"buffer = {buffer:.6f}")
            
        elif action == 'deleted':
            # Clear global variables
            lat = None
            lon = None
            buffer = None
            
            coords_output.value = '<div style="padding: 10px; background-color: white; border-radius: 5px;">' + \
                                'Draw a rectangle to get coordinates</div>'
            
            print("Global variables have been cleared")
    
    # Add draw control
    draw_control = DrawControl(
        rectangle={'shapeOptions': {'color': '#0000FF', 'fillOpacity': 0.2}},
        polygon={},
        circlemarker={},
        circle={},
        marker={},
        polyline={},
        edit=True,
        remove=True
    )
    draw_control.on_draw(handle_draw)
    m.add_control(draw_control)
    return m


# Specify output destinations
outdir = os.path.join(gdata_dir, "Data/PadSeg/")
tmp_dir = scratch_dir

### View the area being downloaded and update if needed

In [2]:

# Choosing location
# lat, lon = -34.3890427, 148.469499
# lat, lon = -35.200628, 149.145974
# buffer=0.045
# stub = "CRGU3"
# start_year = 2020
# end_year = 2021

# stub="ACT-N-01-2021"
lat=-35.245652
lon=149.094576
buffer=0.1209
# start_year = 2021
# end_year = 2021

lat = -35.414108
lon = 149.018555
buffer = 0.297148
stub="ACT-all"
start_year = 2021
end_year = 2021

lat=-35.105305
lon=149.091339
buffer=0.001373

lat=-35.407004
lon=149.078980
buffer=0.288780

m = plot_polygon(lat=lat,lon=lon,start_year=start_year,end_year=end_year,stub=stub,buffer=buffer)
print(f"Check if the polygon for stub='{stub}' is positioned correctly.\nIf it looks good then continue. \nOtherwise, draw a new polygon on the map, update the lat/lon/buffer and run this cell again.")

# Display the map
display(m)

Check if the polygon for stub='ACT-all' is positioned correctly.
If it looks good then continue. 
Otherwise, draw a new polygon on the map, update the lat/lon/buffer and run this cell again.


Map(center=[-35.407004, 149.07898], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [48]:
print(tmp_dir)

/scratch/xe2/tbb801


### Non time-series layers:

In [ ]:
%%time
# Download elevation from terrain tiles
terrain_tiles(lat, lon, buffer, outdir, stub, tmp_dir)

terrain tiles filename = /scratch/xe2/tbb801/ACT-all_terrain_original.tif
Downloaded /scratch/xe2/tbb801/ACT-all_terrain_original.tif


In [4]:
%%time
# The CSIRO soils API seems to frequently be randomly unavailable unfortunately
variables = ['Clay', 'Sand', 'Silt', 'pH_CaCl2']
slga_soils(variables, lat, lon, buffer, outdir, stub)

Downloaded /g/data/xe2/tbb801/Data/PadSeg/ACT-all_Clay.tif
Downloaded /g/data/xe2/tbb801/Data/PadSeg/ACT-all_Sand.tif
Downloaded /g/data/xe2/tbb801/Data/PadSeg/ACT-all_Silt.tif
Downloaded /g/data/xe2/tbb801/Data/PadSeg/ACT-all_pH_CaCl2.tif
CPU times: user 186 ms, sys: 16.9 ms, total: 203 ms
Wall time: 50.3 s


In [5]:
%%time
# Download and merge/crop canopy height tiles from their AWS server
canopy_height(lat, lon, buffer, outdir, stub, tmp_dir)

/g/data/xe2/datasets/Global_Canopy_Height_writeable/tiles_global.geojson
Downloaded: /g/data/xe2/datasets/Global_Canopy_Height_writeable/311230302.tif
Downloaded: /g/data/xe2/datasets/Global_Canopy_Height_writeable/311230213.tif
/g/data/xe2/datasets/Global_Canopy_Height_writeable/tiles_global.geojson
Saved: /g/data/xe2/tbb801/Data/PadSeg/ACT-all_canopy_height.tif
CPU times: user 1min 57s, sys: 41.2 s, total: 2min 38s
Wall time: 18min 23s


### Time-series layers:
(These may require smaller areas to be processed successfully) 

In [38]:
%%time
# Download average climate data from SILO hosted on OzWald
variables = ["Tmax", "Tmin", "Pg"]
ozwald_yearly_average(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

Saved: /g/data/xe2/tbb801/Data/PadSeg/riverview-2_Maximum_Temperature_2017_2023_average.tif
Saved: /g/data/xe2/tbb801/Data/PadSeg/riverview-2_Minimum_Temperature_2017_2023_average.tif
Saved: /g/data/xe2/tbb801/Data/PadSeg/riverview-2_Annual_Rainfall_2017_2023_average.tif
CPU times: user 621 ms, sys: 310 ms, total: 931 ms
Wall time: 6.2 s


#### The 8-day climate modelling data requires access to the Ozwald project on NCI (ub8)

In [40]:
%%time
# Download modelled variables from OzWald
variables = ['Ssoil']
end_year = 2024
ds = ozwald_8day(variables, lat, lon, buffer, start_year, end_year, outdir, stub)
#def ozwald_8day(variables=["Ssoil", "GPP"], lat=-34.3890427, lon=148.469499, buffer=0.01, start_year="2020", end_year="2021", outdir=scratch_dir, stub="Test"):


Saved: /g/data/xe2/tbb801/Data/PadSeg/riverview-2_ozwald_8day.nc
CPU times: user 1.93 s, sys: 587 ms, total: 2.51 s
Wall time: 4.71 s


###
---


In [41]:
%%time
# Download daily weather data hosted on OzWald
print("Seems to fail sporadically, so if process fails, try running again. \nAlso doesn't seem to be able to handle current year so if you get a \nweird error about xarrays, try change the 'end_year' to last year.")

variables = ["VPeff", "Uavg"]
ds = ozwald_daily(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

Downloaded /scratch/xe2/tbb801/riverview-2_VPeff_2017.nc
Downloaded /scratch/xe2/tbb801/riverview-2_VPeff_2018.nc
Downloaded /scratch/xe2/tbb801/riverview-2_VPeff_2019.nc
Downloaded /scratch/xe2/tbb801/riverview-2_VPeff_2020.nc
Downloaded /scratch/xe2/tbb801/riverview-2_VPeff_2021.nc
Downloaded /scratch/xe2/tbb801/riverview-2_VPeff_2022.nc
Downloaded /scratch/xe2/tbb801/riverview-2_VPeff_2023.nc
Downloaded /scratch/xe2/tbb801/riverview-2_VPeff_2024.nc


ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'rasterio', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [42]:
%%time
# Load data pre-downloaded to gdata from SILO's AWS server
ds_silo_daily = silo_daily(["daily_rain", "max_temp", "min_temp", "et_morton_actual", "et_morton_potential"], lat, lon, buffer, start_year, end_year, outdir, stub)
ds_silo_daily

Saved: /g/data/xe2/tbb801/Data/PadSeg/riverview-2_silo_daily.nc
CPU times: user 766 ms, sys: 12.3 s, total: 13 s
Wall time: 59 s


<xarray.Dataset>
Dimensions:              (lat: 1, lon: 2, time: 2788)
Coordinates:
  * lat                  (lat) float64 -35.55
  * lon                  (lon) float64 149.1 149.1
  * time                 (time) datetime64[ns] 2017-01-01 ... 2024-08-19
Data variables:
    daily_rain           (time, lat, lon) float32 0.0 0.0 0.0 ... 0.2 1.8 1.1
    crs                  (time) object b'' b'' b'' b'' b'' ... b'' b'' b'' b''
    max_temp             (time, lat, lon) float32 25.6 26.8 26.3 ... nan nan
    min_temp             (time, lat, lon) float32 13.0 13.4 13.0 ... 2.2 3.6 4.2
    et_morton_actual     (time, lat, lon) float32 2.5 2.8 5.6 ... nan nan nan
    et_morton_potential  (time, lat, lon) float32 4.4 4.5 7.8 ... nan nan nan
Attributes:
    department:               Department of Environment and Science
    department_short:         DES
    copyright:                Copyright - the State of Queensland Department ...
    site_url:                 http://www.longpaddock.qld.gov.au
    institution:              Queensland Government, Department of Environmen...
    raster_source:            Gridded surface was created by interpolating ob...
    raster_source_additions:  and other suppliers (see the SILO webpage for d...
    metadata_url:             http://qldspatial.information.qld.gov.au/catalo...
    reference:                Jeffrey, S.J., Carter, J.O., Moodie, K.B.A. and...
    disclaimer:               1. The user accepts all responsibility and risk...